In [1]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================
# HRR / VSA UTILITY FUNCTIONS
# ==========================

def normalize(v):
    return v / np.linalg.norm(v)

def make_vec(dim=512):
    """Create a random normalized high-dimensional vector."""
    return normalize(np.random.randn(dim))

def bind(a, b):
    """Circular convolution (HRR binding)."""
    return np.fft.ifft(np.fft.fft(a) * np.fft.fft(b)).real

def inverse(a):
    """Compute the inverse for unbinding (conjugate in Fourier domain)."""
    return np.fft.ifft(np.conj(np.fft.fft(a))).real

def bundle(*vectors):
    """Superpose vectors (HRR bundling)."""
    v = np.sum(vectors, axis=0)
    return normalize(v)

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [2]:
dim = 512

# Base concepts
EVEN = make_vec(dim)
ODD = make_vec(dim)
BLUE = make_vec(dim)
GREEN = make_vec(dim)

# Negated versions
NOT = make_vec(dim)   # used to bind NOT with a concept

def negate(v):
    return -v  # simplest representation

# Structural roles
ANT = make_vec(dim)
REL = make_vec(dim)
CONS = make_vec(dim)
IMPLIES = make_vec(dim)

In [3]:
# Rule 1: blue → even
R1 = bundle(
    bind(ANT, BLUE),
    bind(REL, IMPLIES),
    bind(CONS, EVEN)
)

A1_star = bundle(
    BLUE,
    bind(NOT, negate(EVEN))
)

# Rule 2: odd → green
R2 = bundle(
    bind(ANT, ODD),
    bind(REL, IMPLIES),
    bind(CONS, GREEN)
)

A2_star = bundle(
    ODD,
    bind(NOT, negate(GREEN))
)

rules = [R1, R2]
targets = [A1_star, A2_star]

In [4]:
import torch
import torch.nn as nn

class ReasoningNet(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * 2),
            nn.ReLU(),
            nn.Linear(dim * 2, dim)
        )
    
    def forward(self, x):
        return self.net(x)

# 訓練
model = ReasoningNet(dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

rules_t = torch.tensor(np.stack(rules), dtype=torch.float32)
targets_t = torch.tensor(np.stack(targets), dtype=torch.float32)

for epoch in range(100):
    pred = model(rules_t)
    # cosine similarity loss
    loss = 1 - torch.nn.functional.cosine_similarity(pred, targets_t).mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.9912
Epoch 20, Loss: 0.0040
Epoch 40, Loss: 0.0005
Epoch 60, Loss: 0.0001
Epoch 80, Loss: 0.0000


In [5]:
# 測試新規則
RED = make_vec(dim)

R_test = bundle(
    bind(ANT, RED),
    bind(REL, IMPLIES),
    bind(CONS, ODD)
)

A_test_star = bundle(
    RED,
    bind(NOT, negate(ODD))
)

R_test_t = torch.tensor(R_test, dtype=torch.float32).unsqueeze(0)
pred_test = model(R_test_t).detach().numpy().squeeze()

sim = cosine_sim(pred_test, A_test_star)
print(f"Cosine similarity with expected result for new rule: {sim:.4f}")

Cosine similarity with expected result for new rule: -0.0226
